## Getting Started With LangChain

Working directly with OpenAI API can be overwhelming. For example, the API doesn't remember anything from the previous request. You have to create and update a list of messages to keep track of the conversation. This is just one of the many things that LangChain handles for you.

LangChain is a framework for developing applications powered by language models. It provides abstractions for common tasks such as memory, document retrieval, prompt templates, etc. This notebook will walk you through the basics of LangChain.

## 1. Install LangChain

1. In the [Quickstart](01-quickstart.ipynb) notebook, you created a project directory. Open a terminal and navigate to that directory.

2. Activate your virtual environment.

3. Install LangChain.

    ```
    pip install langchain
    ```

## 2. LLMs

In LangChain, you can work with an `LLM` object, which takes a string as an input and returns a string. LangChain provides built-in functions for working with LLMs. For now, just focus on `predict` method.

In [1]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict("What is disaster preparedness?")
response

'\n\nDisaster preparedness is the process of preparing for the potential impacts of a disaster so that the effects can be minimized. This includes making a plan for evacuation, stocking up on supplies, creating an emergency contact list, and maintaining a disaster kit with essential items. It also includes learning about the types of disasters that may occur and being aware of the local emergency management system.'

## 3. Chat Models

Besides from `LLM`, LangChain also support `ChatModel` object. The input here is a list of messages and the output is a `ChatMessage` object.

A `ChatMessage` has two required components:

- `content`: This is the content of the message.
- `role`: This is the role of the entity from which the `ChatMessage` is coming from.

LangChain provides several objects to easily distinguish between different roles, but you only need to worry about the following:

- `HumanMessage`: A `ChatMessage` coming from a human/user.
- `AIMessage`: A `ChatMessage` coming from an AI/assistant.

Although the `predict` method also works with `ChatModel`, it is recommended to use `predict_messages` method instead.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

chat_model = ChatOpenAI()

chat_history = [
    HumanMessage(
        content="I want you to act as a disaster risk reduction and management expert."
    ),
    AIMessage(
        content="I will act as a disaster risk reduction and management expert."
    ),
    HumanMessage(
        content="What is disaster preparedness? Define the concept in two brief sentences."
    ),
]

response = chat_model.predict_messages(chat_history)
response

AIMessage(content='Disaster preparedness refers to the actions taken before a disaster occurs to minimize its impact, including the development of plans, systems, and resources to effectively respond and recover from the event. It involves assessing risks, training personnel, and implementing measures to enhance resilience and readiness for potential disasters.')

The `predict_messages` method doesn't directly return a string. Instead, it returns an `AIMessage` object that wraps the output string. To get the string, you have to access it using the `content` attribute.

In [3]:
response.content

'Disaster preparedness refers to the actions taken before a disaster occurs to minimize its impact, including the development of plans, systems, and resources to effectively respond and recover from the event. It involves assessing risks, training personnel, and implementing measures to enhance resilience and readiness for potential disasters.'

## 4. Prompt Templates

Most LLM applications do not pass user input directly into an LLM. Usually they will add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand. `PromptTemplate` objects help with exactly this! They bundle up all the logic for going from user input into a fully formatted prompt.

In [6]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "What is {concept}? Define the concept in two brief sentences.",
)

prompt.format(concept="disaster preparedness")

'What is disaster preparedness? Define the concept in two brief sentences.'

With `PromptTemplate`, you can:

- Format only some of the variables at a time
- Combine different templates into a single prompt
- Use the same template for different LLMs
- And more!

Complementing the `predict_messages` method of `ChatModel`, LangChain provides `ChatPromptTemplate` object to produce a list of `ChatMessageTemplate` objects. Each `ChatMessageTemplate` contains instructions for how to format that ChatMessage - its role, and then also its content. Let's take a look at this below:

In [9]:
from langchain.prompts.chat import ChatPromptTemplate

template1 = "I want you to act as a {profession}."
template2 = "I will act as a {profession}."
template3 = "What is {concept}? Define the concept in two brief sentences."

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", template1),
        ("ai", template2),
        ("human", template3),
    ]
)

chat_history = chat_prompt.format_messages(
    profession="disaster risk reduction and management expert",
    concept="disaster preparedness",
)

chat_history

[HumanMessage(content='I want you to act as a disaster risk reduction and management expert.'),
 AIMessage(content='I will act as a disaster risk reduction and management expert.'),
 HumanMessage(content='What is disaster preparedness? Define the concept in two brief sentences.')]

We can now use the list of `ChatMessageTemplate` objects as an input to `ChatModel.predict_messages`.

In [11]:
chat_model = ChatOpenAI()
response = chat_model.predict_messages(chat_history)
response.content

'Disaster preparedness is the proactive process of planning, organizing, and implementing measures to minimize the impact of potential disasters. It involves developing strategies, systems, and resources to effectively respond and recover from disasters, ensuring the safety and well-being of communities.'